Célula 1: Importações e Configurações

In [ ]:
!pip install pandas
!pip install numpy
!pip install scikit-learn
!pip install matplotlib
!pip install seaborn
!pip install scipy
!pip install scikit-image
!pip install scikit-learn
!pip install scikit-learn

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier # Opcional
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pickle # Para salvar o scaler e o modelo
import os # Para garantir que os caminhos de arquivo funcionem bem
from sklearn.tree import export_text # Para exportar regras da árvore de decisão

# --- Configurações ---
INPUT_FEATURES_CSV = 'dataset_with_features.csv'
MODEL_CHOICE = 'logistic' # Escolha: 'logistic', 'svm_linear', 'decision_tree'
TEST_SIZE_RATIO = 0.2   # Proporção do dataset para o conjunto de teste (ex: 0.2 para 20%)
RANDOM_STATE_SEED = 42  # Para reprodutibilidade

# Caminhos para salvar o scaler e o modelo treinado
SCALER_FILE_PATH = 'scaler.pkl'
MODEL_FILE_PATH = 'trained_model.pkl'
MODEL_PARAMS_FILE_PATH = 'model_parameters.txt' # Para salvar pesos e bias em formato de texto
# ---------------------

print("Configurações carregadas.")

Célula 2: Função para Salvar Parâmetros do Modelo para C/C++

In [ ]:
def save_model_parameters_for_c(model, scaler, model_type, feature_names_list_param, file_path):
    """Salva os parâmetros do modelo e do scaler em um arquivo de texto para fácil implementação em C/C++."""
    with open(file_path, 'w') as f:
        f.write(f"// --- Parâmetros do Modelo ({model_type}) e Scaler para Implementação em C/C++ ---\n")
        f.write(f"// Gerado em: {pd.Timestamp.now()}\n")
        
        f.write("\n// Parâmetros do StandardScaler (média e escala/desvio_padrão):\n")
        f.write("// Use estes para escalar as features no ESP32 antes da predição\n")
        f.write(f"// Número de features que o scaler espera: {len(scaler.mean_)}\n")
        f.write("// Nomes das features (na ordem esperada pelo scaler/modelo):\n")
        
        if feature_names_list_param:
            for i, name in enumerate(feature_names_list_param):
                f.write(f"// Feature {i}: {name}\n")
        else:
            f.write("// Nomes das features não disponíveis. Ordem é crucial.\n")

        f.write("const float scaler_means[] = {")
        for i, mean_val in enumerate(scaler.mean_):
            f.write(f"{mean_val:.8f}f")
            if i < len(scaler.mean_) - 1:
                f.write(", ")
        f.write("};\n")

        f.write("const float scaler_scales[] = {") # scale_ é o desvio padrão para StandardScaler
        for i, scale_val in enumerate(scaler.scale_):
            f.write(f"{scale_val:.8f}f")
            if i < len(scaler.scale_) - 1:
                f.write(", ")
        f.write("};\n")

        if model_type in ['logistic', 'svm_linear']:
            # model.coef_ é geralmente [[w1, w2, ...]] para classificação binária
            weights = model.coef_[0]
            bias = model.intercept_[0]
            f.write(f"\n// Parâmetros do Modelo ({model_type}):\n")
            f.write("// y_pred_raw = w[0]*f_scaled[0] + ... + w[n-1]*f_scaled[n-1] + bias\n")
            if model_type == 'logistic':
                 f.write("// P(y=1) = 1 / (1 + exp(-y_pred_raw)); Prever 1 se P(y=1) > 0.5\n")
            else: # svm_linear
                 f.write("// Prever 1 se y_pred_raw > 0 (ou outro limiar de decisão)\n")
            
            f.write("const float model_weights[] = {")
            for i, weight in enumerate(weights):
                f.write(f"{weight:.8f}f")
                if i < len(weights) - 1:
                    f.write(", ")
            f.write("};\n")
            f.write(f"const float model_bias = {bias:.8f}f;\n")
            
            # Imprimir no console do notebook também
            print("\n--- Parâmetros do Modelo e Scaler para Implementação em C/C++ ---")
            if feature_names_list_param:
                print(f"Nomes das Features (ordem): {feature_names_list_param}")
            print(f"Scaler Means: {scaler.mean_}")
            print(f"Scaler Scales (Std Devs): {scaler.scale_}")
            print(f"Model Weights: {weights}")
            print(f"Model Bias: {bias}")
        
        elif model_type == 'decision_tree':
            f.write("\n// Parâmetros da Árvore de Decisão:\n")
            f.write("// A exportação direta de uma árvore para C requer uma lógica de if/else.\n")
            f.write("// Use sklearn.tree.export_text(model, feature_names=...) para ver as regras.\n")
            tree_rules = export_text(model, feature_names=feature_names_list_param)
            f.write("\n// Regras da Árvore de Decisão (para referência):\n")
            f.write(tree_rules + "\n")
            print("\nRegras da Árvore de Decisão (para referência):")
            print(tree_rules)
        
        print(f"\nParâmetros e/ou regras do modelo salvos em texto em: {file_path}")

print("Função save_model_parameters_for_c definida.")

Célula 3: Carregamento e Pré-processamento dos Dados


In [ ]:
print(f"--- Iniciando Treinamento do Modelo ({MODEL_CHOICE}) ---")

# 1. Carregar Dados
try:
    df_features = pd.read_csv(INPUT_FEATURES_CSV)
    print(f"Dataset de features lido: {INPUT_FEATURES_CSV} ({len(df_features)} janelas, {len(df_features.columns)-1} features + label)")
except FileNotFoundError:
    print(f"ERRO: Arquivo de features não encontrado: {INPUT_FEATURES_CSV}")
    print("Certifique-se de que o script 'feature_extractor.py' foi executado com sucesso.")
    # Em um notebook, você pode querer parar aqui ou carregar um df de exemplo
    df_features = None 
except Exception as e:
    print(f"ERRO ao ler {INPUT_FEATURES_CSV}: {e}")
    df_features = None

if df_features is not None:
    if 'label' not in df_features.columns:
        print(f"ERRO: A coluna 'label' não foi encontrada em {INPUT_FEATURES_CSV}.")
        df_features = None # Impede a execução do resto se o label não existir
    
    # Tratamento de NaNs
    if df_features.isnull().sum().any():
        print("AVISO: Valores nulos (NaN) encontrados no dataset de features.")
        print("Preenchendo NaNs numéricos com a média da coluna...")
        for col in df_features.columns:
            if df_features[col].isnull().any() and pd.api.types.is_numeric_dtype(df_features[col]):
                if col != 'label': # Não preenche a coluna de label
                    df_features[col] = df_features[col].fillna(df_features[col].mean())
        
        if df_features.isnull().sum().any():
            print("AVISO: Ainda há NaNs após o preenchimento com média. Removendo linhas com NaNs restantes.")
            df_features.dropna(inplace=True)
            print(f"Novo tamanho do dataset após remoção de NaNs: {len(df_features)}")
            if len(df_features) == 0:
                print("Dataset vazio após remoção de NaNs. Saindo.")
                df_features = None
    
    if df_features is not None:
        X = df_features.drop('label', axis=1)
        y = df_features['label']
        feature_names = list(X.columns) # Salva os nomes das features

        print(f"\nNúmero de amostras: {len(X)}")
        print(f"Número de features: {len(feature_names)}")
        print(f"Nomes das Features: {feature_names}")
        print(f"Distribuição das classes:\n{y.value_counts(normalize=True)}")
        
        # Visualizar as primeiras linhas de X e y
        print("\nPrimeiras 5 linhas das Features (X):")
        print(X.head())
        print("\nPrimeiras 5 linhas dos Rótulos (y):")
        print(y.head())
else:
    print("Não foi possível carregar ou processar os dados. Verifique as mensagens de erro.")

Célula 4: Divisão Treino/Teste e Escalonamento


In [ ]:
if 'X' in locals() and 'y' in locals(): # Verifica se X e y foram definidos na célula anterior
    # 2. Dividir em Treino e Teste
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, 
        test_size=TEST_SIZE_RATIO, 
        random_state=RANDOM_STATE_SEED, 
        stratify=y # Importante para manter a proporção das classes em treino e teste
    )
    print(f"\nDataset dividido em {len(X_train)} amostras de treino e {len(X_test)} de teste.")

    # 3. Escalonamento de Features
    scaler = StandardScaler()
    # Ajusta o scaler APENAS nos dados de treino
    X_train_scaled = scaler.fit_transform(X_train)
    # Aplica a mesma transformação aos dados de teste
    X_test_scaled = scaler.transform(X_test) 
    
    # Converte de volta para DataFrames para manter nomes de colunas (opcional, mas bom para inspeção)
    X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=feature_names)
    X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=feature_names)
    
    print("Features escalonadas usando StandardScaler.")
    print("\nPrimeiras 5 linhas das Features de Treino Escalonadas:")
    print(X_train_scaled_df.head())

    # Salva o scaler treinado
    with open(SCALER_FILE_PATH, 'wb') as f_scaler:
        pickle.dump(scaler, f_scaler)
    print(f"\nScaler treinado salvo em: {SCALER_FILE_PATH}")
else:
    print("X e/ou y não foram definidos. Execute a célula de carregamento de dados primeiro.")

Célula 5: Escolha, Treinamento e Salvamento do Modelo


In [ ]:
if 'X_train_scaled' in locals() and 'y_train' in locals(): # Verifica se os dados de treino existem
    # 4. Escolher e Treinar o Modelo
    print(f"\nTreinando modelo: {MODEL_CHOICE}...")
    if MODEL_CHOICE == 'logistic':
        model = LogisticRegression(solver='liblinear', random_state=RANDOM_STATE_SEED, class_weight='balanced')
    elif MODEL_CHOICE == 'svm_linear':
        model = SVC(kernel='linear', probability=True, random_state=RANDOM_STATE_SEED, class_weight='balanced')
    elif MODEL_CHOICE == 'decision_tree':
        model = DecisionTreeClassifier(random_state=RANDOM_STATE_SEED, max_depth=5, class_weight='balanced') 
    else:
        print(f"ERRO: Escolha de modelo inválida: {MODEL_CHOICE}.")
        model = None # Impede a execução do resto se o modelo for inválido

    if model is not None:
        model.fit(X_train_scaled, y_train) # Usa X_train_scaled (numpy array)
        print("Modelo treinado.")

        # Salva o modelo treinado
        with open(MODEL_FILE_PATH, 'wb') as f_model:
            pickle.dump(model, f_model)
        print(f"Modelo treinado salvo em: {MODEL_FILE_PATH}")
else:
    print("Dados de treino (X_train_scaled, y_train) não encontrados. Execute as células anteriores.")

Célula 6: Avaliação do Modelo no Conjunto de Teste


In [ ]:
if 'model' in locals() and 'X_test_scaled' in locals() and 'y_test' in locals(): # Verifica se tudo necessário existe
    # 5. Avaliação no Conjunto de Teste
    print("\n--- Avaliação no Conjunto de Teste ---")
    y_pred_test = model.predict(X_test_scaled) # Usa X_test_scaled (numpy array)
    
    accuracy_test = accuracy_score(y_test, y_pred_test)
    print(f"Acurácia no Teste: {accuracy_test:.4f}")
    
    print("\nMatriz de Confusão (Teste):")
    cm = confusion_matrix(y_test, y_pred_test)
    print(cm)
    # Para visualização mais clara da matriz de confusão:
    # import seaborn as sns
    # import matplotlib.pyplot as plt
    # sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=model.classes_, yticklabels=model.classes_)
    # plt.xlabel("Previsto")
    # plt.ylabel("Real")
    # plt.title("Matriz de Confusão - Teste")
    # plt.show()

    print("\nRelatório de Classificação (Teste):")
    # Certifique-se de que target_names corresponda às suas classes e à ordem em model.classes_
    # Se y é 0 e 1, e model.classes_ é [0, 1], então os nomes são para 0 e depois para 1.
    class_names = [f'NaoTremor ({model.classes_[0]})', f'Tremor ({model.classes_[1]})'] if len(model.classes_)==2 else None
    print(classification_report(y_test, y_pred_test, target_names=class_names))
else:
    print("Modelo ou dados de teste não encontrados. Execute as células anteriores.")


Célula 7: Validação Cruzada


In [ ]:
if 'model' in locals() and 'X_train_scaled' in locals() and 'y_train' in locals():
    # 6. Validação Cruzada (no conjunto de treino)
    print("\n--- Validação Cruzada (no conjunto de treino escalonado) ---")
    # Usa X_train_scaled e y_train
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='accuracy') # 5 folds
    print(f"Acurácias da Validação Cruzada (5-fold): {cv_scores}")
    print(f"Acurácia Média da Validação Cruzada: {np.mean(cv_scores):.4f} (+/- {np.std(cv_scores):.4f})")
else:
    print("Modelo ou dados de treino não encontrados para validação cruzada.")

Célula 8: Salvar Parâmetros do Modelo e Scaler para C/C++


In [ ]:
if 'model' in locals() and 'scaler' in locals() and 'feature_names' in locals():
    # 7. Salvar Parâmetros do Modelo para C/C++
    # Passar 'feature_names' que guardamos de X antes de escalar.
    # A função 'save_model_parameters_for_c' foi definida na Célula 2.
    save_model_parameters_for_c(model, scaler, MODEL_CHOICE, feature_names, MODEL_PARAMS_FILE_PATH)
    print("\nScript de treinamento de modelo (execução no notebook) concluído.")
else:
    print("Modelo, scaler, ou feature_names não definidos. Não foi possível salvar os parâmetros para C.")
